There is a lack of affordable access to many important medications in the Arab world. Some of these medicationsa are available in neighboring countries at reasonable prices. If these medications can be distributed effectively taking advantage of existing transport between the countries, then these crucial medications can be made more plentiful and affordable. To minimize the cost of medications and take advantage of this opportunity for arbitrage, transport of medications from airports to consumers must be as efficient as possible. We solve this problem as a vehicle routing problem where the objective is to minimize the overall cost of deliveries given a fleet of vehicles with limited capacities. We formulate this as a VQE algorithm which is solved using quantum annealing with D-Wave's superconducting hardware.

In [2]:
from dimod import (
    Binary,
    BinaryQuadraticModel,
    ConstrainedQuadraticModel,
    quicksum,
)

from dwave.system import LeapHybridBQMSampler

import random
import pandas as pd
import numpy as np


# Insert your token here to run or create a file credentials.py to track it
#TOKEN = ""

from credentials import TOKEN

First, consider the case where there is a single airport where medication can be delivered and the medication must be delivered at the lowest cost possible by a fleet of delivery vehicles. These vehicles are limited in capacity and how far they can travel in one day. To solve this problem efficiently with VQE, we first formalize it as a constrained optimization problem:

Consider when there are $M$ vehicles and $N$ delivery locations. 

Let $x_{i, j, k}$ be an indicator variable such that $x_{i, j, k} = 1$ if vehicle $i$ visits dropoff location $j$ at stop number $k$ on its route and $x_{i, j, k} = 0$ otherwise.  

In [3]:
def build_vrp_bqm(num_locations, distances, num_vehicles, max_distance):
    """
    Input:
        num_locations: number of distinct locations for orders
        distances: an matrix with the distances between respective locations for a fully connected graph
        num_vehicles: the number of vehicles used
        max_distance: the maximum distance that a truck can drive in one day
    """

    M = num_vehicles
    N = num_locations

    SINGLE_LOCATION_CONSTANT = 10**7
    VISIT_ALL_CONSTANT = 10**7

    # Create all the variables: one for each vehicle/location/position combo
    # k is position, j is vertex, i is vehicle
    x = {(i, j, k): Binary(str(i) + "_" + str(j) + "_" + str(k)) for k in range(N+1) for j in range(N+1) for i in range(M)}

    print(x)

    print(f"{x}\n")

    # Define the unconstrained binary optimization problem
    obj = BinaryQuadraticModel(vartype="BINARY")

    # The cost of going from the depot to the first stop
    for m in range(M):
        for n in range(N):
            obj += x[m, n, 0] * distances[N][n]

    # The cost of going from the last stop to the depot
    for m in range(M):
        for n in range(N):
            obj += x[m, n, N-1] * distances[n][N]

    # The cost of going between all stops in the middle
    for m in range(M):
        for n in range(N-1):
            for i in range(N+1):
                for j in range(N+1):
                    obj += x[m, i, n] * x[m, j, n + 1] * distances[i][j]

    
    # 1. Each location should be served by exactly one vehicle (only checks first N because depot is otherwise factored in)
    for k in range(N):
        # A term
        for a in range(M):
            for b in range(N):
                for c in range(M):
                    for d in range(N):
                        if a != b or c != d:
                            e = x[a, k, b]
                            f = x[c, k, d]
                            obj += x[a, k, b] * x[c, k, d] * VISIT_ALL_CONSTANT
                obj -= 0.5 * x[a, k, b] * VISIT_ALL_CONSTANT   

    # 2. Each vehicle is in at most one location (this accounts for how some paths will not hit all vertices)
    for m in range(M):
        for n in range(N):
            # A term
            for i in range(N+1):
                for j in range(i+1, N+1):
                    obj += SINGLE_LOCATION_CONSTANT * 2 * x[m, i, n] * x[m, j, n]
                obj -= SINGLE_LOCATION_CONSTANT * x[m, i, n]
                            
    # Return the unconstrained optimization solution
    return obj

In [4]:
def run_bqm(bqm):
    """Run the provided CQM on the Leap Hybrid CQM Sampler."""
    sampler = LeapHybridBQMSampler(token=TOKEN)

    sampleset = sampler.sample_bqm(bqm)
    feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)

    num_feasible = len(feasible_sampleset)
    errors = " "
    if num_feasible == 0:
        print("No feasible solution found.")
        return sampleset

    print("\nFeasible solution found.\n")

    return feasible_sampleset

In [5]:
# Create a sample problem
num_destinations = 5
num_vehicles = 2
max_distance = 30

# Generate a random symmetric cost matrix
cost_matrix = [[0]*(num_destinations+1) for _ in range(num_destinations + 1)]

for i in range(num_destinations + 1):
    for j in range(i, num_destinations + 1):
        if i == j:
            cost_matrix[i][j] = 0
        else:
            val = random.randint(6,10)
            cost_matrix[i][j] = val
            cost_matrix[j][i] = val

bqm = build_vrp_bqm(num_destinations, cost_matrix, num_vehicles, max_distance)

{(0, 0, 0): BinaryQuadraticModel({'0_0_0': 1.0}, {}, 0.0, 'BINARY'), (1, 0, 0): BinaryQuadraticModel({'1_0_0': 1.0}, {}, 0.0, 'BINARY'), (0, 1, 0): BinaryQuadraticModel({'0_1_0': 1.0}, {}, 0.0, 'BINARY'), (1, 1, 0): BinaryQuadraticModel({'1_1_0': 1.0}, {}, 0.0, 'BINARY'), (0, 2, 0): BinaryQuadraticModel({'0_2_0': 1.0}, {}, 0.0, 'BINARY'), (1, 2, 0): BinaryQuadraticModel({'1_2_0': 1.0}, {}, 0.0, 'BINARY'), (0, 3, 0): BinaryQuadraticModel({'0_3_0': 1.0}, {}, 0.0, 'BINARY'), (1, 3, 0): BinaryQuadraticModel({'1_3_0': 1.0}, {}, 0.0, 'BINARY'), (0, 4, 0): BinaryQuadraticModel({'0_4_0': 1.0}, {}, 0.0, 'BINARY'), (1, 4, 0): BinaryQuadraticModel({'1_4_0': 1.0}, {}, 0.0, 'BINARY'), (0, 5, 0): BinaryQuadraticModel({'0_5_0': 1.0}, {}, 0.0, 'BINARY'), (1, 5, 0): BinaryQuadraticModel({'1_5_0': 1.0}, {}, 0.0, 'BINARY'), (0, 0, 1): BinaryQuadraticModel({'0_0_1': 1.0}, {}, 0.0, 'BINARY'), (1, 0, 1): BinaryQuadraticModel({'1_0_1': 1.0}, {}, 0.0, 'BINARY'), (0, 1, 1): BinaryQuadraticModel({'0_1_1': 1.0},

In [6]:
feasible_sampleset = run_bqm(bqm)

AttributeError: 'LeapHybridSampler' object has no attribute 'sample_bqm'

In [ ]:
def parse_string(input_string):
    return list(map(int, input_string.split('_')))

def build_routes_from_sample(sample, num_vehicles):
    """Builds a set of routes from the sample returned."""

    routes =  [[] for _ in range(num_vehicles)]

    # Go through all entries
    for key, val in sample.items():
        vehicle, vertex, step = parse_string(key)
        if val == 1.0:
            if len(routes[vehicle]) < 1 or vertex != routes[vehicle][-1]:
                routes[vehicle].append(vertex)

    return routes

def build_routes_from_sample_raw(sample, num_vehicles):
    """Builds a set of routes from the sample returned."""

    routes =  [[] for _ in range(num_vehicles)]

    # Go through all entries
    for key, val in sample.items():
        vehicle, vertex, step = parse_string(key)
        if val == 1.0:
            routes[vehicle].append(vertex)

    return routes

def get_cost_from_sample(paths, distances):
    cost = 0

    for path in paths:
        if len(path) >= 2:
            for i in range(len(path) - 1):
                cost += distances[path[i]][path[i+1]]
            cost += distances[0][path[0]]
            cost += distances[path[len(path) - 1]][len(distances) - 1]
            

    return cost
            



In [ ]:
lowest_energy_sample = feasible_sampleset.lowest().first.sample

routes = build_routes_from_sample(lowest_energy_sample, num_vehicles)
raw_routes = build_routes_from_sample_raw(lowest_energy_sample, num_vehicles)

print(routes)
print(raw_routes)
print(feasible_sampleset.lowest().first.energy)

cost = get_cost_from_sample(routes, cost_matrix)
print(cost)

print(feasible_sampleset)

[[2, 4, 6, 8, 9], [0, 10, 1, 3, 5, 7]]
[[2, 4, 6, 8, 9], [0, 10, 1, 3, 5, 7]]
0.0
89
   0_0_0 0_0_1 0_0_10 0_0_2 0_0_3 0_0_4 0_0_5 ... 1_9_9 energy num_oc. ...
0    0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
1    0.0   1.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
2    0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
3    0.0   1.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
4    0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
5    0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
6    0.0   1.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
7    0.0   1.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
8    0.0   1.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
9    0.0   1.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
10   0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
11   0.0   0.0 

In [ ]:
min_cost = None
min_path = []

# A very quick and dirty benchmark
for i in range(100):
    array = list(range(num_destinations))
    values = np.random.permutation(array)
    split = random.randint(0, num_destinations)
    path = [values[:split], values[split:]]

    cost = get_cost_from_sample(path, cost_matrix)

    if not min_cost or cost < min_cost:
        min_cost = cost
        min_path = path

print(min_cost)
print(min_path)

72
[array([6, 4, 0, 1, 2, 8, 3, 9, 7]), array([5])]
